In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [11]:
! pip install langchain-community langchain_openai faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 31.4 MB/s eta 0:00:00


## Retrieval Augmented Generation (RAG)

### Carregando Documentos - Loading

In [4]:
# https://python.langchain.com/v0.2/docs/how_to/#document-loaders
# https://python.langchain.com/v0.2/docs/integrations/document_loaders/

import bs4
from langchain_community.document_loaders import WebBaseLoader

# Filtra o conteúdo da página por uma classe específica
bs4_strainer = bs4.SoupStrainer(class_=("container-wrapper"))

# Carrega o conteúdo da página
loader = WebBaseLoader(
    web_paths=("https://cesar.breezy.hr/p/00f79174d8ad-pesquisador-em-inteligencia-artificial-e-sistemas-distribuidos",),
    bs_kwargs={"parse_only": bs4_strainer},
)

# Carrega o conteúdo da página
docs = loader.load()

len(docs[0].page_content)

4301

### Dividindo Documentos - Splitting/Chunking

In [5]:
# https://python.langchain.com/v0.2/docs/how_to/#text-splitters

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=500, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

6

In [6]:
print(all_splits[3].page_content)

Requisitos e Qualificações:
Doutorado em áreas correlatas;Compreenda e desenvolva modelos de machine learning e deep learning para resolver desafios complexos de cibersegurança;Conhecimento em frameworks de machine learning como TensorFlow, PyTorch ou scikit-learn para desenvolver modelos preditivos e de detecção de anomalias aplicados à cibersegurança;Entenda a arquitetura distribuída dos sistemas e garanta a integração eficiente de soluções de IA com aplicações em cloud;Habilidade em manipulação e visualização de dados com Pandas, NumPy, Matplotlib e Seaborn para explorar grandes volumes de dados;Experiência com AWS, Google Cloud ou Azure para projetar e implementar infraestruturas escaláveis e resilientes;Familiaridade com Kubernetes e Docker para garantir escalabilidade e resiliência de sistemas distribuídos;Experiência com controle de versão (Git) e repositórios remotos como GitLab;Inglês avançado para leitura, escrita e comunicação, facilitando a colaboração com equipes globais.


### Indexando - Store

In [12]:
# https://python.langchain.com/v0.2/docs/how_to/embed_text/

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

vectorstore = FAISS.from_documents(all_splits, OpenAIEmbeddings())

In [13]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("precisa de doutorado para a vaga?")

len(retrieved_docs)

6

In [14]:
print(retrieved_docs[0].page_content)

Requisitos e Qualificações:
Doutorado em áreas correlatas;Compreenda e desenvolva modelos de machine learning e deep learning para resolver desafios complexos de cibersegurança;Conhecimento em frameworks de machine learning como TensorFlow, PyTorch ou scikit-learn para desenvolver modelos preditivos e de detecção de anomalias aplicados à cibersegurança;Entenda a arquitetura distribuída dos sistemas e garanta a integração eficiente de soluções de IA com aplicações em cloud;Habilidade em manipulação e visualização de dados com Pandas, NumPy, Matplotlib e Seaborn para explorar grandes volumes de dados;Experiência com AWS, Google Cloud ou Azure para projetar e implementar infraestruturas escaláveis e resilientes;Familiaridade com Kubernetes e Docker para garantir escalabilidade e resiliência de sistemas distribuídos;Experiência com controle de versão (Git) e repositórios remotos como GitLab;Inglês avançado para leitura, escrita e comunicação, facilitando a colaboração com equipes globais.


### Buscando e Recuperando Informações - Retrieve

In [15]:
from langchain_core.prompts import ChatPromptTemplate

system_template = """Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

In [16]:
example_messages = prompt_template.invoke({
    "context": "algum contexto",
    "question": "alguma pergunta"
})

print(example_messages.to_messages())

[HumanMessage(content='Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.\n\nPergunta: alguma pergunta\n\nContexto: algum contexto\n\nResposta:\n', additional_kwargs={}, response_metadata={})]


### Gerando Respostas - Generate

In [17]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [18]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [19]:
for chunk in rag_chain.stream("Tem plano de saúde como benefício?"):
    print(chunk, end="", flush=True)

Sim, o CESAR oferece plano de saúde como parte de seu pacote de benefícios. Além disso, também disponibiliza plano odontológico e outros auxílios.

## Exercícios

### Exercício 1
Faça um RAG com um pequeno arquivo de texto, contendo informações que, certamente, a LLM não conheça. Você deverá construir o arquivo e enviar para o ambiente de execução. Escolha a forma de chunking apropriada para o seu documento.

In [39]:
! pip install pypdf wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=e9a65bf07cc168bc31d1d95669b19d5ae28486d714e818eaa7ee483a9ab1acbb
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [51]:
from langchain_community.document_loaders import PyPDFLoader
import wget

In [64]:
url = 'https://nbviewer.org/github/thaisrezendeb/topicos_contemporaneos_202401/blob/main/Part%202%20-%20LLMs/historia_fogueira.pdf'
wget.download(url, 'historia_fogueira.pdf')

loader = PyPDFLoader('/content/historia_fogueira.pdf')
pages = loader.load_and_split() # Faz a leitura e o chunk

print(pages[0].page_content)

Home/ História da Fogueira
História da Fogueira
Prefeitura Municipal de Cachoeira de Minas
 
 
A Festa da Fogueira de São Pedro, segundo aponta a versão mais difundida na cidade, teve origem por volta do ano de 1958,
com a iniciativa do Sr. Pedro Ribeiro, conhecido como Pedro da Colodina. No começo eram pequenos festejos juninos em
homenagem a São Pedro, tendo como foco principal do evento a queima de uma pequena fogueirinha montada em um terreno
vago, atual Praça do Rosário, no Bairro do Rosário, próximo ao centro de Cachoeira de Minas. Após o falecimento do Sr.Pedro da
Coldina estas movimentações se encerraram
A Fogueira de São Pedro teve início com um forte contexto religioso. Pedro era mascate e visitava a cidade periodicamente,
principalmente no período da Festa de São João Batista. Devoto do santo, Pedro da Colodina possuía uma deficiência física e
usava latinhas de extrato de tomate forradas com espuma para apoiar suas mãos e se locomover.
Ele fez uma promessa que realizaria uma

In [67]:
# Store
vectorstore = FAISS.from_documents(pages, OpenAIEmbeddings())
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
retrieved_docs = retriever.invoke("Qual o tamanho da maior fogueira?")

In [70]:
# Usando o mesmo template definido anteriormente, agora com o novo conteúdo (retriever)
rag_chain_ex1 = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [83]:
for chunk in rag_chain_ex1.stream("Em qual cidade/estado é a fogueira? Em que lugar ela é construída? Quantos anos tem a tradição?"):
    print(chunk, end="", flush=True)

A fogueira é construída na cidade de Cachoeira de Minas, em Minas Gerais, na Praça do Rosário. A tradição da Festa da Fogueira de São Pedro teve início por volta de 1958.